# **GitHub Commands**

In [1]:
# @title GitHub Init
# from google.colab import userdata

# GH_UNAME = userdata.get('GH_UNAME')
# GH_APIKEY = userdata.get('GH_APIKEY')
# GH_EMAIL = userdata.get('GH_EMAIL')
PRIMARY_REPO_NAME = 'Legal-Research-Platform'
LOCAL_REPO_DIR = '/content/drive/MyDrive/FYP/GitHub/Legal-Research-Platform'

# !git config --global user.name {GH_UNAME}
# !git config --global user.email {GH_EMAIL}

# %cd {LOCAL_REPO_DIR}

In [2]:
# @title Git <-
!git fetch

!git pull

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


In [3]:
# @title Git ->
# !git add .

# !git status

# !git commit -m 'updated layout'

!git push

fatal: not a git repository (or any of the parent directories): .git


# **Scrapers**

In [ ]:
#@title law acts scraper

# Step 1: Install required libraries
# !apt-get update
# !apt-get purge chromium-browser chromium-chromedriver -y
# !apt-get autoremove -y
# !wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
# !dpkg -i google-chrome-stable_current_amd64.deb || apt-get -fy install
# !pip install -U selenium webdriver-manager requests

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Clean output
from IPython.display import clear_output
clear_output()

# Step 2: Import required libraries
import os
import time
import shutil
import logging
import requests
import math
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from concurrent.futures import ThreadPoolExecutor, as_completed

# Step 4: Define download paths
local_download_dir = "/content/downloads"
drive_directory = "/content/drive/MyDrive/FYP/legal_acts_raw"  # Replace with your desired directory

# Ensure directories exist
os.makedirs(local_download_dir, exist_ok=True)
os.makedirs(drive_directory, exist_ok=True)

# Step 5: Set up Selenium WebDriver
chrome_options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": local_download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True,
}
chrome_options.add_experimental_option("prefs", prefs)
chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-gpu")  # Disable GPU hardware acceleration
chrome_options.add_argument("--window-size=1920x1080")  # Use a fixed window size
chrome_options.add_argument("--disable-dev-shm-usage")

# Initialize WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# Create a session with a larger connection pool
session = requests.Session()
adapter = HTTPAdapter(pool_connections=100, pool_maxsize=100, max_retries=Retry(total=2))
session.mount("http://", adapter)
session.mount("https://", adapter)


# Process each row separately (to be used in threading)
def process_row(row, year):
    try:
        #Get Name
        name = row.find_element(By.CSS_SELECTOR, "td:nth-child(3)").text.strip()

        # Find download links inside <a> tags that contain buttons
        english_link = row.find_element(By.XPATH, ".//a[button[contains(text(), 'English')]]")
        sinhala_link = row.find_element(By.XPATH, ".//a[button[contains(text(), 'Sinhala')]]")

        # Get the actual download URLs
        english_url = english_link.get_attribute("href") if english_link else None
        sinhala_url = sinhala_link.get_attribute("href") if sinhala_link else None

        # Download files in parallel
        if english_url:
            download_file(english_url, f"{name}_English.pdf", year)
        if sinhala_url:
            download_file(sinhala_url, f"{name}_Sinhala.pdf", year)

    except Exception as e:
        print(f"Error processing row for year {year}: {e}")

# iterative function to process row chunks
def process_rows_iterative(rows, year, max_threads=20):
    futures = []
    while rows:
        num_rows = len(rows)
        num_threads = min(max_threads, max(1, num_rows // 2))
        chunk_size = math.ceil(num_rows / num_threads)
        row_chunks = [rows[i:i + chunk_size] for i in range(0, num_rows, chunk_size)]

        print(f"Processing {num_rows} rows with {num_threads} threads, chunk size: {chunk_size}")

        with ThreadPoolExecutor(max_workers=num_threads) as executor:
            futures = [executor.submit(process_row, row, year) for chunk in row_chunks for row in chunk]

        # Update remaining rows
        rows = rows[chunk_size * num_threads:]

    # Wait for all threads to complete before proceeding
    for future in as_completed(futures):
        try:
            future.result()
        except Exception as e:
            print(f"Error in thread: {e}")



# Step 6: Define the scraper function
def scrape_legal_acts(url):
    try:
        driver.get(url)
        time.sleep(3)
        print("Browser Opened")

        # Find all year buttons
        year_buttons = driver.find_elements(By.XPATH, "//a[@class='btn btn-primary']")
        print(f"Found {len(year_buttons)} year buttons")

        for i in range(len(year_buttons)):
            button = driver.find_elements(By.XPATH, "//a[@class='btn btn-primary']")[i]
            year = button.text.strip()
            print(f"Processing year: {year}")

            button.click()
            time.sleep(3)

            if not driver.find_elements(By.CSS_SELECTOR, "table tbody tr"):
                print(f"No data found for year: {year}")
                driver.back()
                time.sleep(2)
                continue

            # Find all rows in the table
            rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")
            print(f"Found {len(rows)} rows for year: {year}")

            # Process each row in parallel
            process_rows_iterative(rows, year)

            # Return to the year selection page
            driver.back()
            time.sleep(2)

    except Exception as e:
        print(f"Error during scraping: {e}")


# Step 7: Download files function
def download_file(url, filename, year):
    try:
        response = session.get(url, stream=True)
        if response.status_code == 200:
            year_folder = os.path.join(drive_directory, year)
            os.makedirs(year_folder, exist_ok=True)
            filepath = os.path.join(year_folder, filename)

            with open(filepath, "wb") as file:
                for chunk in response.iter_content(chunk_size=1024):
                    file.write(chunk)

            print(f"Downloaded: {filename}")
        else:
            print(f"Failed to download {filename}")

        response.close()

    except Exception as e:
        print(f"Error downloading file {filename}: {e}")

# Step 8: Run the scraper
website_url = "https://documents.gov.lk/view/acts/acts.html"  # Replace with the actual URL
scrape_legal_acts(website_url)

# Step 9: Close the WebDriver
driver.quit()

Browser Opened
Found 45 year buttons
Processing year: 2025
No data found for year: 2025
Processing year: 2024
Found 32 rows for year: 2024
Processing 32 rows with 16 threads, chunk size: 2


Downloaded: Saweera Foundation (Incorporation)_English.pdf
Downloaded: Shop and Office Employees (Regulation of Employment and Remuneration) (Amendment)_English.pdf
Downloaded: International Institute of Theravadha (Incorporation)_English.pdf
Downloaded: Public Debt Management_English.pdf
Downloaded: Partition (Amendment)_English.pdf
Downloaded: Saweera Foundation (Incorporation)_Sinhala.pdf
Downloaded: Shop and Office Employees (Regulation of Employment and Remuneration) (Amendment)_Sinhala.pdf
Downloaded: International Institute of Theravadha (Incorporation)_Sinhala.pdf
Downloaded: Public Debt Management_Sinhala.pdf


Downloaded: Partition (Amendment)_Sinhala.pdf
Downloaded: Social Security Contribution Levy (Amendment)_English.pdf
Downloaded: Value Added Tax (Amendment)_English.pdf
Downloaded: Heart to Heart Trust Fund (Incorporation)_English.pdf
Downloaded: Sri Lanka Baptist Sangamaya (Incorporation) (Amendment)_English.pdf
Downloaded: Sri Balabhivurdhi Wardana Society (Incorporation)_English.pdf


Downloaded: Social Security Contribution Levy (Amendment)_Sinhala.pdf
Downloaded: Value Added Tax (Amendment)_Sinhala.pdf
Downloaded: Heart to Heart Trust Fund (Incorporation)_Sinhala.pdf
Downloaded: Sri Lanka Baptist Sangamaya (Incorporation) (Amendment)_Sinhala.pdf


Downloaded: Sri Balabhivurdhi Wardana Society (Incorporation)_Sinhala.pdf
Downloaded: Samadhi Community Development Foundation (Incorporation)_English.pdf
Downloaded: Dassana Bauddha Sanvidhanaya (Incorporation)_English.pdf
Downloaded: National Hydrographic_English.pdf
Downloaded: Contempt of a Court,Tribunal or Institution_English.pdf
Downloaded: Online Safety_English.pdf
Downloaded: Samadhi Community Development Foundation (Incorporation)_Sinhala.pdf
Downloaded: Dassana Bauddha Sanvidhanaya (Incorporation)_Sinhala.pdf


...

Downloaded: Appropriation (Amendment)_English.pdf
Downloaded: Registration of Electors (Amendment)_English.pdf
Downloaded: Termination of Employment of Workmen (Special Provisions) (Amendment)_English.pdf
Downloaded: Employees Provident Fund (Amendment)_English.pdf
Downloaded: Appropriation (Amendment)_Sinhala.pdf
Downloaded: Registration of Electors (Amendment)_Sinhala.pdf
Downloaded: Termination of Employment of Workmen (Special Provisions) (Amendment)_Sinhala.pdf
Downloaded: Employees Provident Fund (Amendment)_Sinhala.pdf
Downloaded: Securities and Exchange Commission of Sri Lanka_English.pdf
Downloaded: Securities and Exchange Commission of Sri Lanka_Sinhala.pdf
Downloaded: National Minimum Wage of Workers (Amendment)_English.pdf
Downloaded: National Minimum Wage of Workers (Amendment)_Sinhala.pdf


Downloaded: Inland Revenue (Amendment)_Sinhala.pdf


Downloaded: Bail (Amendment)_English.pdf
Downloaded: Bail (Amendment)_Sinhala.pdf
Processing year: 2020
Found 8 rows for year: 2020
Processing 8 rows with 4 threads, chunk size: 2


Downloaded: Finance (Amendment)_English.pdf
Downloaded: 20th Amendment to the Constitution (NEW)_English.pdf
Downloaded: Appropriation_English.pdf
Downloaded: Appropriation_English.pdf
Downloaded: Finance (Amendment)_Sinhala.pdf
Downloaded: 20th Amendment to the Constitution (NEW)_Sinhala.pdf
Downloaded: Nation Building Tax (Amendment)_English.pdf
Downloaded: Appropriation_Sinhala.pdf
Downloaded: Economic Service Charge (Amendment)_English.pdf
Downloaded: Appropriation_Sinhala.pdf
Downloaded: ports and Airports Development Levy (Amendment)_English.pdf
Downloaded: Nation Building Tax (Amendment)_Sinhala.pdf
Downloaded: Economic Service Charge (Amendment)_Sinhala.pdf
Downloaded: ports and Airports Development Levy (Amendment)_Sinhala.pdf
Downloaded: Institute of Environmental Professionals, Sri Lanka (Incorporation)_English.pdf
Downloaded: Institute of Environmental Professionals, Sri Lanka (Incorporation)_Sinhala.pdf
Processing year: 2019
Found 24 rows for year: 2019
Processing 24 rows 

KeyboardInterrupt: 

In [ ]:
#@title case download

import json
import requests
import os
from datetime import datetime
import nest_asyncio  # For environments like Jupyter notebooks

# Apply nest_asyncio to allow nested event loops if needed
nest_asyncio.apply()

# Specify the path to your JSON file
file_path = "/content/drive/MyDrive/FYP/resources/jurilens-db.documents.json"  # Replace with the actual path to your JSON file

# Read the JSON file and load its data
with open(file_path, 'r') as file:
    json_data = json.load(file)  # Load JSON data as a Python dictionary

# Define the download function
def download_pdf(file_info, save_location):
    pdf_url = file_info['url']
    pdf_source_url = file_info['sourceUrl']
    pdf_name = file_info['name']

    # Make a request to download the PDF
    if pdf_url:
      response = requests.get(pdf_url)
    else:
      print("falling back to source url")
      response = requests.get(pdf_source_url)

    if response.status_code == 200:
        # Create the full path for saving the file
        save_path = os.path.join(save_location, pdf_name)

        # Write the content to a PDF file
        with open(save_path, 'wb') as pdf_file:
            pdf_file.write(response.content)

        print(f"Downloaded and saved: {pdf_name}")
    else:
        print(f"Failed to download PDF: {response.status_code}")

# Specify the base location where you want to save the downloaded PDFs
base_save_location = "/content/drive/MyDrive/FYP/law_cases_raw"  # Change this to your desired base path

# Process each entry in the JSON data
for entry in json_data[664:1000]:
    if 'file' in entry and 'date' in entry:
        file_info = entry['file']

        # Extract year from date (assuming date is in ISO format)
        year = entry['date']['$date'][:4]  # Get the first four characters representing the year

        # Create a directory for the year if it doesn't exist
        year_folder = os.path.join(base_save_location, year)
        os.makedirs(year_folder, exist_ok=True)  # Create directory if it doesn't exist

        # Download the PDF and save it with its given name in the year folder
        download_pdf(file_info, year_folder)

falling back to source url
Downloaded and saved: wrt_0203_20_001_pdf.pdf
Downloaded and saved: rii_23_2022_judgment_pdf.pdf
Downloaded and saved: hcc_165_13_judgment_pdf.pdf
Downloaded and saved: coc_04_19_judgment_pdf.pdf
Downloaded and saved: hcc_0023_21_pdf.pdf
Downloaded and saved: ca_hcc_91_20_judgmentn_pdf.pdf
Downloaded and saved: ca_writ_410_2017_pdf.pdf
Downloaded and saved: ca_phc_106_17_pdf.pdf
Downloaded and saved: cpa_0011_20_final_judgment_pdf.pdf
Downloaded and saved: wrt_0437_20_1_pdf.pdf
Downloaded and saved: ca_hcc_231_235_15_judgment_pdf.pdf
Downloaded and saved: hcc_0109_18_pdf.pdf
Downloaded and saved: hcc_0355_357_18_pdf.pdf
Downloaded and saved: ca_writ_340_2019_2023_06_22_pdf.pdf
Downloaded and saved: ca_writ_64_2019_2023_06_22_pdf.pdf
Downloaded and saved: ca_writ_0430_19_pdf.pdf
Downloaded and saved: ca_wrt_0333_21_001_pdf.pdf
Downloaded and saved: bail_order_cpa_0016_23_pdf.pdf
Downloaded and saved: hcc_0373_0374_19_pdf.pdf
Downloaded and saved: hcc_0191_15_p

# Preprocessing

In [4]:
#@title Init

# Install required packages and dependencies
!pip install PyPDF2 pdf2image pytesseract
!apt-get install -y poppler-utils
!apt-get install -y tesseract-ocr tesseract-ocr-sin tesseract-ocr-tam

!pip install fasttext
!pip install --upgrade googletrans==4.0.2
!wget -O lid.176.bin https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin

#mount drive
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.0 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.6 [186 kB]
Fetched 186 kB in 1s (183 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 124926 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.6_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.6) ...
Setting up poppler-utils (22.02.0-2ubuntu0.6) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The followin

In [ ]:
import os
import uuid
from PyPDF2 import PdfReader
from pdf2image import convert_from_path
import pytesseract
from concurrent.futures import ThreadPoolExecutor, as_completed

def extract_text_from_pdf(pdf_path):
    """Attempts to extract text from a PDF using PyPDF2."""
    text = ""
    try:
        reader = PdfReader(pdf_path)
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {e}")
    return text.strip()

def ocr_pdf(pdf_path, lang="eng+sin"):
    """Uses pdf2image to convert PDF pages to images and then applies OCR."""
    text = ""
    try:
        images = convert_from_path(pdf_path)
        for img in images:
            page_text = pytesseract.image_to_string(img, lang=lang)
            text += page_text + "\n"
    except Exception as e:
        print(f"Error OCR processing {pdf_path}: {e}")
    return text.strip()

def process_pdf(pdf_path, lang="eng+sin", text_threshold=10):
    """Attempts to extract text from PDF and falls back to OCR if needed."""
    text = extract_text_from_pdf(pdf_path)
    if len(text) < text_threshold:
        print(f"Text extraction yielded very little text for {pdf_path.split('/').pop()}. Running OCR...")
        text = ocr_pdf(pdf_path, lang=lang)
    return text

def process_pdf_file(pdf_path, lang="eng+sin"):
    """Processes a single PDF file and returns its data as a dictionary."""
    unique_id = str(uuid.uuid4())
    filename = os.path.basename(pdf_path)
    text = process_pdf(pdf_path, lang=lang)

    # Assuming language is part of the filename
    primary_lang = filename.split('_')[1].split('.')[0]

    return {
        "id": unique_id,
        "filename": filename.split('_')[0],
        "primaryLang": primary_lang,
        "text": text,
        "wordCount": len(text.split()),
    }

def process_folder(folder_path, lang="eng+sin", max_workers=1):
    """Iterates over all PDF files in a folder and processes them."""
    results = []
    pdf_files = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path) if filename.lower().endswith(".pdf")]

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(process_pdf_file, pdf_path, lang): pdf_path for pdf_path in pdf_files}

        for future in as_completed(futures):
            pdf_path = futures[future]
            try:
                result = future.result()  # Get result of the future
                results.append(result)
                print(f"Processed: {result['filename']} | Word Count: {result['wordCount']}")  # Change 'length' to 'wordCount'
            except Exception as e:
                print(f"Error processing {pdf_path}: {e}")

    return results

# -----------------------------
# Usage Example
# -----------------------------
folder_path = "/content/drive/MyDrive/FYP/legal_acts_raw/1991"  # Update this path as needed

# Process all PDFs in the folder
pdf_data = process_folder(folder_path)

# Now, pdf_data is a list of dictionaries containing 'id', 'filename', and 'text'
for data in pdf_data:
    print(f"ID: {data['id']}\nFilename: {data['filename']}\nPrimary Language: {data['primaryLang']}\nWord Count: {data['wordCount']}\nText Preview: {data['text'][:200]}\n{'-'*50}")

In [ ]:
#@title Translation

import fasttext
import asyncio
from googletrans import Translator
import nest_asyncio  # For environments like Jupyter notebooks

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

# Load FastText model
model = fasttext.load_model("lid.176.bin")

# Initialize Google Translate API
translator = Translator()

async def detect_language_fasttext(text, word_threshold=300):
    """Detects if the text contains non-English content."""
    words = text.split()
    total_words = len(words)
    num_chunks = max(1, total_words // word_threshold)

    for i in range(num_chunks):
        chunk = " ".join(words[i * word_threshold:(i + 1) * word_threshold])
        prediction = model.predict(chunk)
        detected_lang = prediction[0][0].replace("__label__", "")
        if detected_lang != "en":
            print("Needs Translation")
            return True  # Indicates that translation is needed

    return False  # The text is already in English

async def translate_if_needed(text):
    """Translates the whole text to English asynchronously if needed."""
    if await detect_language_fasttext(text):
        try:
            # Run the synchronous translator.translate call in a separate thread
            loop = asyncio.get_event_loop()
            translation = await loop.run_in_executor(None, translator.translate, text, 'en')
            tr = await translation
            return tr.text
        except Exception as e:
            print("Translation error: " + str(e))
            return text  # Return the original text if an error occurs
    else:
        return text  # No translation needed

async def process_documents(pdf_data):
    """Processes documents asynchronously."""
    for doc in pdf_data:
        original_text = doc.get("text", "")
        print(f"Checking document: {doc['filename']}")
        translated_text = await translate_if_needed(original_text)
        doc["text"] = translated_text

    # Print a preview of the updated text
    for doc in pdf_data:
        print(f"ID: {doc['id']}\nFilename: {doc['filename']}\nText Preview: {doc['text'][:200]}\n{'-'*50}\n")

# Main function to run process_documents
async def main():
    await process_documents(pdf_data)

# Run the main function in an environment with an existing event loop
try:
    loop = asyncio.get_running_loop()  # Get the current running loop
except RuntimeError:  # No running event loop, create a new one
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)

# Await the main function (ensuring all tasks finish)
if loop.is_running():
    # For environments like Jupyter Notebook
    task = asyncio.create_task(main())
    await task  # Wait for completion of the task
else:
    loop.run_until_complete(main())
